In [1]:
import sys
sys.path.append('..')

## GPU 지원

In [3]:
#!pip install cupy

In [ ]:
# Jupyter Notebook에서는 cupy 모듈이 인식되지 않음
import cupy as cp
import numpy as np


x = cp.arange(6).reshape(2, 3)
y = np.array([1, 2, 3])

# NumPy to CuPy
a = cp.asarray(y)
print(a)
print(type(a))
print(type(y))
assert type(a) == cp.ndarray

# CuPy to NumPy
b = cp.asnumpy(x)
print(b)
print(type(b))
print(type(x))
assert type(b) == np.ndarray

xp = cp.get_array_module(y)
assert xp == np
xp = cp.get_array_module(x)
assert xp == cp

In [ ]:
# cuda.py - Update

### Variable/Layer/DataLoader Class Update

In [ ]:
class Variable:
    """ Update Version (Level 5-1).
    Class Variable : np.array 값을 다루되 다른 멤버변수를 추가적인 특징으로 가져 다양한 정보를 모두 포함시키는 클래스.

    Parameter
    ---------
    data : np.array
        실제 연산에서 사용되는 값으로, 다양한 연산 및 정보 전달에도 사용
    grad : int, float
        역전파 수행 시, 현재 역전파 층에서의 gradient 값을 저장하여 다음 역전파에 전달하는 기능을 수행
    creator : callable function
        Variable 객체(인스턴스)를 연산하기 위한 함수가 무엇인지 저장하여 현재 객체는 creator에 들어온 함수값을 수행하기 위해 존재하는 것으로 판단하면 됨
    generation : Variable 객체가 어느 세대(generation)에 포함되어 있는지를 표현하기 위한 변수

    Functions
    ---------
    # 재귀 방식과 반복문 방식의 경우 메모리 할당 측면에서 반복문 방식이 유리하다.
    # 재귀 방식은 호출 시마다 메모리에 누적되나 반복문의 경우 pop을 활용해 메모리가 누적되지 않은 상태로 작업을 수월한다.
    __init__
        isinstance : 첫 번째 파라미터값의 타입과 두 번째 파라미터값의 동등 여부를 확인하는 함수
        retain_grad : gradient 값(y().grad)을 유지시킬지 설정하는 변수. default = False
        name : 차후 Variable에 이름을 달아주기 위해 설정
        >> Update
        : array_types라는 tuple에 모듈 type을 저장한 후 해당 타입을 init에서 체크
    cleargrad
        메모리 절약을 위해서 한 번 할당했던 변수를 또 다시 사용해야 할 경우 기존 메모리에 있던 Variable.grad 정보가 남아있기 때문에 할당을 해제해주어야 제대로 동작을 수행하게 된다.
    backward
        반복문을 이용한 역전파 코드
        while loop 내에서 zip 함수를 활용하여 x.grad에 중복되게 값이 들어갈 경우 기존 노드에 더해지도록 만들어 x + x = 2x 라는 식을 예로 들었을 때, gradient 값이 2로 정상 출력되게 하였음
        add_func
            역전파 시 함수가 pop되면서 새로운 값이 들어올 때, 같은 메모리를 참조하는 경우에 중복 방지를 위해서 추가된 함수
            이미지 결과를 보면 더 이해가 쉽기 때문에 아래 코드블럭 중 Check로 Markdown 표시를 한 곳에서 이미지를 확인하기를 추천!!!
        weakref
            순환 참조로 인해 생기는 메모리 누수 문제를 해결하기 위해 import된 weakref 객체에 대해 실제로 값을 출력할 때는 Variable처럼 weakref도 어떠한 기본 데이터타입을 감싸고 있는 형태이므로 기본 생성자를 통해 객체를 호출함과 동시에 output을 사용해 실제 결과값을 확인함
        self.grad. 즉 역전파 시작 시 기존에는 ndarray 인스턴스를 받았으나 고차 미분을 가능하게 하기 위해 ndarray 인스턴스가 아닌 Variable 인스턴스를 받도록 설정
        입력 파라미터에 create_graph 변수 추가. 역전파 1회 계산 후 역전파를 비활성 모드로 실행하게 만드는 파라미터
        with using_config(name, value) 구문을 생성하여 역전파 설정을 통해 들여쓰기된 구문의 수행 여부를 판단
        >> Update
        : myPackage.cuda.get_array_module로 데이터타입 체크 후 import할 모듈 선택하는 구문 추가
    shape, ndim, size, dtype : numpy에 기본적으로 내장되어있는 메서드를 @property 데코레이터를 활용하여 바로 호출할 수 있도록 설정
    __len__ : data의 길이 반환
    __repr__ : print로 객체를 표현할 때 return할 값을 설정
    __mul__ : 다른 객체 또는 데이터타입과의 multiply 기능 지원
    reshape(self, *shape) : 만약에 가변인자로 들어오는 shape 값이 1개인 경우 그 값의 instance가 tuple or list일 때 shape[0]를 shape로 지정하고 그 외의 경우에는 myPackage.functions.reshape 함수를 사용하여 *shape의 값을 그대로 반영하여 reshape 진행
    transpose : Variable 인스턴스에서 transpose 메서드를 호출했을 때, myPackage.functions에서 transpose 함수를 바로 호출할 수 있도록 설정
    T : transpose를 바로 실행할 수 있도록 만든 @property function. @property 데코레이터를 사용하여 self 객체를 instance 변수로 바로 사용할 수 있도록 설정하였음
    sum : myPackage.functions.sum 함수를 호출하여 Variable 인스턴스에서 바로 sum 함수를 호출할 수 있도록 설정
    >> Update
        to_gpu & to_cpu: Variable 데이터를 GPU or CPU로 전송해주는 기능을 수행하는 메서드
    """
    def __init__(self, data, name = None):
        if data is not None:
            if not isinstance(data, array_types):
                raise TypeError(f'{type(data)} is not supported.')

        self.data = data
        self.name = name
        self.grad = None
        self.creator = None
        self.generation = 0

    def cleargrad(self):
        self.grad = None

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1

    @property
    def shape(self):
        return self.data.shape

    @property
    def ndim(self):
        return self.data.ndim

    @property
    def size(self):
        return self.data.size

    @property
    def dtype(self):
        return self.data.dtype

    @property
    def T(self):
        return myPackage.functions.transpose(self)

    def __len__(self):
        return len(self.data)

    def __repr__(self):
        if self.data is None:
            return "variable(None)"
        p = str(self.data).replace('\n', '\n' + ' ' * 9)
        return f"variable({p})"

    def reshape(self, *shape):
        if len(shape) == 1 and isinstance(shape[0], (tuple, list)):
            shape = shape[0]
        return myPackage.functions.reshape(self, shape)

    def transpose(self):
        return myPackage.functions.transpose(self)

    def sum(self, axis=None, keepdims=False):
        return myPackage.functions.sum(self, axis, keepdims)

    def backward(self, retain_grad = False, create_graph = False):
        if self.grad is None:
            xp = myPackage.cuda.get_array_module(self.data)
            self.grad = Variable(xp.ones_like(self.data))

        funcs = []
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key = lambda x: x.generation)

        add_func(self.creator)

        while funcs:
            f = funcs.pop()
            gys = [output().grad for output in f.outputs]

            with using_config('enable_backprop', create_graph):
                gxs = f.backward(*gys)
                if not isinstance(gxs, tuple):
                    gxs = (gxs,)

                for x, gx in zip(f.inputs, gxs):
                    if x.grad is None:
                        x.grad = gx
                    else:
                        x.grad = x.grad + gx

                    if x.creator is not None:
                        add_func(x.creator)
            
            if not retain_grad:
                for y in f.outputs:
                    y().grad = None

    def to_cpu(self):
        if self.data is not None:
            self.data = myPackage.cuda.as_numpy(self.data)

    def to_gpu(self):
        if self.data is not None:
            self.data = myPackage.cuda.as_cupy(self.data)

In [ ]:
class Layer:
    """
    Layer class
    -----------
    input, hidden, output layer에 대한 기반클래스

    Args
    ----
    name: Layer's name.
    value: Layer's value is put in name's attritube.
    inputs: This parameter means layer's input data.

    Method
    ------
    __setattr__: Get parameter(name, value) and check a value's type in Parameter instance. Then, set attribute value about name.
    __call__: Make callable function instance with inputs value(get variable argument)(This method use weakref.ref for help to circular reference.).
    forward: It will be necessary method from derived class.
    params: Yield a layer instances value using iterator.
    cleargrad: Clear all value's gradient.
    >> Update
        to_gpu & to_cpu: Variable 데이터를 GPU or CPU로 전송해주는 기능을 수행하는 메서드
    """
    def __init__(self):
        self._params = set()

    def __setattr__(self, name, value):
        if isinstance(value, (Parameter, Layer)): # Layer 추가
            self._params.add(name)
        super().__setattr__(name, value)

    def __call__(self, *inputs):
        outputs = self.forward(*inputs)
        if not isinstance(outputs, tuple):
            outputs = (outputs,)
        self.inputs = [weakref.ref(x) for x in inputs]
        self.outputs = [weakref.ref(y) for y in outputs]
        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, inputs):
        raise NotImplementedError()

    def params(self):
        for name in self._params:
            obj = self.__dict__[name]

            if isinstance(obj, Layer): # Layer에서 매개변수 꺼내기
                yield from obj.params()
            else:
                yield obj
        
    def cleargrads(self):
        for param in self.params():
            param.cleargrad()

    def to_cpu(self):
        for param in self.params():
            param.to_cpu()

    def to_gpu(self):
        for param in self.params():
            param.to_gpu()

In [ ]:
class DataLoader:
    """
    DataLoader class

    Args
    ----
    dataset : Dataset 인터페이스를 만족하는 인스턴스(이 말인 즉슨, __getitem__ & __len__ 메서드를 구현한 클래스로부터 생성된 인스턴스를 의미)
    batch_size : 배치 크기
    shuffle : 에포크별로 데이터셋을 뒤섞을지 여부
    """
    def __init__(self, dataset, batch_size, shuffle=True, gpu=False):
        self.dataset = dataset
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.data_size = len(dataset)
        self.max_iter = math.ceil(self.data_size / batch_size)
        self.gpu = gpu # Update

        self.reset()

    def reset(self):
        self.iteration = 0
        if self.shuffle:
            self.index = np.random.permutation(len(self.dataset))
        else:
            self.index = np.arange(len(self.dataset))

    def __iter__(self):
        return self

    def __next__(self):
        if self.iteration >= self.max_iter:
            self.reset()
            raise StopIteration

        i, batch_size = self.iteration, self.batch_size
        batch_index = self.index[i * batch_size:(i + 1) * batch_size]
        batch = [self.dataset[i] for i in batch_index]
        # x = np.array([example[0] for example in batch])
        # t = np.array([example[1] for example in batch])

        # Update
        xp = cuda.cupy if self.gpu else np
        x = xp.array([example[0] for example in batch])
        t = xp.array([example[1] for example in batch])

        self.iteration += 1

        return x, t

    def next(self):
        return self.__next__()


    def to_cpu(self);
        self.gpu = False

    def to_gpu(self):
        self.gpu = True

## 함수 추가 구현